# Transfer Learning on Mnist Dataset

### Importing Libraries

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


### Declearing hyperparameters

In [2]:
batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

### Function to preprocess and run the model

In [3]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

### Prepare the data for test and train

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [7]:
x_train.shape, y_train.shape, x_train_lt5.shape, x_test_lt5.shape

((60000, 28, 28), (60000,), (30596, 28, 28), (5139, 28, 28))

### Creating the model

In [5]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

model = Sequential(feature_layers + classification_layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

### Train the model

In [6]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 32s 1ms/step - loss: 0.1566 - accuracy: 0.9503 - val_loss: 0.0321 - val_accuracy: 0.9895
Epoch 2/5
30596/30596 [==============================] - 25s 804us/step - loss: 0.0445 - accuracy: 0.9871 - val_loss: 0.0124 - val_accuracy: 0.9955
Epoch 3/5
30596/30596 [==============================] - 25s 802us/step - loss: 0.0311 - accuracy: 0.9909 - val_loss: 0.0122 - val_accuracy: 0.9957
Epoch 4/5
30596/30596 [==============================] - 25s 806us/step - loss: 0.0237 - accuracy: 0.9929 - val_loss: 0.0076 - val_accuracy: 0.9969
Epoch 5/5
5139/5139 [==============================] - 1s 149us/step
Test score: 0.012080308810263994
Test accuracy: 0.9961081743240356


### Tranfer learning the same model by freezing some layers

In [8]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 8s 264us/step - loss: 0.2423 - accuracy: 0.9310 - val_loss: 0.0512 - val_accuracy: 0.9821
Epoch 2/5
29404/29404 [==============================] - 8s 263us/step - loss: 0.0777 - accuracy: 0.9760 - val_loss: 0.0345 - val_accuracy: 0.9889
Epoch 3/5
29404/29404 [==============================] - 8s 266us/step - loss: 0.0587 - accuracy: 0.9817 - val_loss: 0.0287 - val_accuracy: 0.9903
Epoch 4/5
29404/29404 [==============================] - 8s 264us/step - loss: 0.0500 - accuracy: 0.9839 - val_loss: 0.0251 - val_accuracy: 0.9918
Epoch 5/5
4861/4861 [==============================] - 1s 150us/step
Test score: 0.024046854571098487
Test accuracy: 0.9913598299026489


Get the entire code on https://keras.io/examples/mnist_transfer_cnn/